## Load library modules

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

default_dir = os.path.dirname(os.path.dirname(os.getcwd()))
os.chdir(default_dir)

# (Mac) Uncomment below two lines if there's an error with PyJulia setup (initial execution may take some time)
# from julia.api import Julia
# jl = Julia(compiled_modules=False)

# Import neccessary modules
from modWorm import sys_paths
from modWorm import network_params as n_params
from modWorm import network_dynamics as n_dyn
from modWorm import network_interactions as n_inter
from modWorm import network_simulations as n_sim

from modWorm import utils

### Load experimental connectome data
- Varshney, Lav R., et al. "Structural properties of the Caenorhabditis elegans neuronal network." PLoS computational biology 7.2 (2011): e1001066.
- Haspel, Gal, and Michael J. O'Donovan. "A perimotor framework reveals functional segmentation in the motoneuronal network controlling locomotion in Caenorhabditis elegans."
- Download link: https://www.wormatlas.org/images/NeuronConnect.xls
- If the file is in old .xls format, make sure to save as .xlsx format

In [ ]:
# filepath example: 'C:\\Users\\xyz\\Downloads\\filename.xlsx'
conn_gap, conn_syn = utils.construct_connectome_Varshney(filepath)

## Load pre-defined Nervous System model

In [ ]:
from modWorm import predefined_classes_nv

celegans_nv = predefined_classes_nv.CelegansWorm_NervousSystem(conn_gap, conn_syn) # Base nervous system model

## Define Stimulation

In [ ]:
PLM_neuron_inds = utils.neuron_names_2_inds(['PLML', 'PLMR'])   # Find neuron indices for C.elegans neurons

In [ ]:
simulation_time = 5                                             # Simulation duration in seconds
simulation_steps = int(simulation_time/celegans_nv.timescale)   # Total number of timesteps (default timestep: 0.01s)
                                                                # Note this should be associated with timescale defined above

input_mat = np.zeros((simulation_steps, celegans_nv.network_Size))   # Create external input for the period of 5 seconds (timesteps * 279 neurons)
input_mat[:, PLM_neuron_inds] = 2000                                 # Inject 2000pA (2nA) into PLML/R (276, 278th neurons)

## Simulate Nervous System

In [ ]:
# Use run_network() function from "network_simulation" module
# RuntimeWarning: overflow encountered is normal due to the activity stabilization in the beginning of simulation

solution_dict = n_sim.run_network(celegans_nv, input_mat)

## Analyze results

In [ ]:
print(solution_dict.keys())             # solution_dict has 3 keys

In [ ]:
v_sol = solution_dict['v_solution'].T   # Un-normalized voltage solution (t x network_Size)

s_sol = solution_dict['s_solution'].T   # Synaptic activity solution     (t x network_Size)
                                        #                                (ranges from 0 - 1)
                                        #                                (0: neuron's synapses not activated)
                                        #                                (1: neuron's synapses full activated)
            
vthmat = solution_dict['v_threshold'].T # Resting voltage solution       (t x network_Size)

delta_v = v_sol - vthmat                # Normalized voltage is voltage displacement from resting voltage

In [ ]:
# Plot the normalized voltages using pcolor

fig = plt.figure(figsize=(7.5, 4))
plt.pcolor(delta_v, cmap='bwr', vmin = -10, vmax = 10)
plt.xlabel("10 ms")
plt.ylabel("Neurons")
plt.ylim(279, 0)
plt.colorbar()